# Mapping Customers and Suppliers to Tradable Securities
A key use-case of the Supply Chain Relationships DataFeed is Customer-to-Supplier Momentum, or the idea that customer returns drive supplier returns. In order to be able to incorporate such a strategy, is it vital to be able to map from a factset_entity_id to a tradable security such as a Ticker. This notebooks will walk through the process of mapping suppliers of a customer to their primary equity identifiers.

## Notice
This example workbook provides queries in SQL formatted for MSSQL and sample python code. It may be necessary to adjust the query language to suit a different SQL DBMS. Sample queries and code should be viewed as possible examples of data calculations and are subject to change. The queries and code provided should be used as a guide to understand how the underlying data items can be used but are not guaranteed to represent the same methodology as the FactSet Workstation or other industry calculations. Queries cannot be guaranteed to be written for performance and/or efficiency.

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import pyodbc
import matplotlib.pyplot as plt
import os


import sys
sys.path.append('../')
import utils.loadsql as loadsql
sys.path.append('/')

#Define Directory of SQL Queries
sql_path = './../SQL Queries/'

## Create DSN Connection

Enter a DSN associated with the MSSQL database containing FactSet Standard DataFeeds.

In [2]:
dsn = 'SDF'
cxn = pyodbc.connect('DSN={dsn_name}'.format(dsn_name = dsn))

## Retrieving Suppliers and their Tickers
While a main use of the **ENT_SCR_RELEVANCE_RANK** table is to be able to identify the importance of a relationship to a given company, another great value of the table is to be able to plug a company into the **ranked_factset_entity_id** field and return all customer and supplier relationships no matter the source. We'll use this table to identify all suppliers of Walmart to then be able to retrieve their Tickers. In the query below, we allow for an input of a date as the Supply Chain DataFeed offers point-in-time data containing start and end dates for when a relationship was valid.

In [3]:
ticker = 'WMT-US'
date = '2019-05-31'

#loadsql is a cover function used to retrieve SQL queries from a directory
q = loadsql.get_sql_q(os.path.join(sql_path,'1.3.1 Identifying Suppliers and their Tickers.sql')
                      ,show=0,connection=dsn).format(sym=ticker,date=date)
print(q)
rel_rank = pd.read_sql(q,cxn)
rel_rank.head(15)

DECLARE @id AS NVARCHAR(50)= 'WMT-US';
DECLARE @date AS DATE= '2019-05-31';
SELECT rr.customer_factset_entity_id, 
       se2.entity_proper_name AS customer_name, 
       rr.supplier_factset_entity_id, 
       se1.entity_proper_name AS supplier_name, 
       tr2.ticker_region AS supplier_ticker
FROM   sym_V1.sym_ticker_region AS tr1
JOIN sym_V1.sym_coverage AS sc1
  ON sc1.fsym_id = tr1.fsym_id
JOIN ent_V1.ent_scr_sec_entity_hist AS seh1
  ON seh1.fsym_id = sc1.fsym_security_id
     AND @date >= seh1.start_date
     AND @date < ISNULL(seh1.end_date, GETDATE())
JOIN ent_v1.ent_scr_relevance_rank AS rr
  ON rr.ranked_factset_entity_id = seh1.factset_entity_id
     AND @date >= rr.start_date
     AND @date < ISNULL(rr.end_date, GETDATE())
JOIN ent_V1.ent_scr_sec_entity_hist AS seh2
  ON seh2.factset_entity_id = rr.supplier_factset_entity_id
     AND @date >= seh2.start_date
     AND @date < ISNULL(seh2.end_date, GETDATE())
JOIN sym_v1.sym_coverage AS sc2
  ON sc2.fsym_id = seh2.fsym_id
  

,customer_factset_entity_id,customer_name,supplier_factset_entity_id,supplier_name,supplier_ticker
0,000YMS-E,"Walmart, Inc.",0B6B4T-E,"C&S Paper Co., Ltd.",002511-CN
1,000YMS-E,"Walmart, Inc.",0GD63R-E,FIBRA Macquarie México,FIBRAMQ12-MX
2,000YMS-E,"Walmart, Inc.",000D8R-E,Campbell Soup Co.,CPB-US
3,000YMS-E,"Walmart, Inc.",0DK7WJ-E,Munters Group AB,MTRS-SE
4,000YMS-E,"Walmart, Inc.",001FXQ-E,"Euronet Worldwide, Inc.",EEFT-US
5,000YMS-E,"Walmart, Inc.",0G4YRS-E,Yihai International Holding Ltd.,1579-HK
6,000YMS-E,"Walmart, Inc.",0JRV8S-E,"Spectrum Brands Holdings, Inc.",SPB-US
7,000YMS-E,"Walmart, Inc.",006DPK-E,SunPower Corp.,SPWR-US
8,000YMS-E,"Walmart, Inc.",000CT9-E,BBX Capital Corp.,BBX-US
9,000YMS-E,"Walmart, Inc.",000F36-E,"Conagra Brands, Inc.",CAG-US


## Next Steps

* **Please check out these notebooks for additional information on the FactSet Supply Chain Relationships DataFeed:** 
    * 1.1 Introduction to FactSet Supply Chain Relationships
    * 1.2 Linking Market IDs to FactSet Entity IDs


* **Documentation for the feeds discussed above can be found within the** [Open:FactSet Marketplace](https://open.factset.com/en-us)
    * [FactSet Supply Chain Relationships](https://open.factset.com/products/factset-supply-chain-relationships/en-us) 
 
 
* **Please direct any training requests or questions to your CTS Implementation Specialist.**